### To determine the contents of a file:

In [ ]:
f = /path/to/opusfile.0
dbs = opusFC.listContents(f)
print(dbs)

### Load the first datablock in dbs: 

In [ ]:
data = opusFC.getOpusData(f, dbs[0])

### Access loaded data and parameters:

In [ ]:
data.x                    # X coordinates
data.parameters['SNM']    # Sample name
data.description          # Description of data object
opusFC.paramDict['SNM']     # String description of SNM

### Access and save visible images:

In [ ]:
images = opusFC.getVisImages(f)

for img in images:
    data = img['image']
    title = img['Title']
    imgext = img['imgext']
    f = "{0}_{1}.{2}".format(f, title, imgext)
    with open(f, 'wb') as fd:
        fd.write(data)